In [1]:
import pandas as pd
from pprint import pprint
import caplot
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_table('data/samples.tsv.gz')

In [5]:
plot = caplot.PCA()
plot.LoadData(df)
plot.Filter('select * from data where "pcaSS1-scores_1" > 0.02 AND "pheno-age" > 20')
plot.Highlight('select * from data')
plot.Configure(['pca-scores_1', 'pca-scores_3', 'pca-scores_6'],
               coloringColumn='pheno-subpopulation',
               coloringStyle='Categorical',
               coloringPalette='Category20',
               nCols=1)
plot.Hover({
    'sampleID': 's',
    'AgeOfSample': 'pheno-age'})
plot.Show()#WithForm()

In [8]:
plot.Show()

KeyboardInterrupt: 